In [2]:
!pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   --------------------- ------------------ 286.7/542.0 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/401.7 kB ? eta -:--:--
   --------------------------------------  399.4/401.7 kB 12.6 MB/s eta 0:00:01
   --------------------------------------- 401.7/401.7 kB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.8 MB 24.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.4/25.8 MB 18.2 MB/s eta 0:00:02
   --- ------------------------------------ 2.1/25.8 MB 17.0 MB/s eta 0:00:02
   ---- ----------------------------------- 2.7/25.8 MB 17.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/25.8 MB 15.9 MB/s eta 0:00:02
   ------ --------------------------------- 4.2/25.8 MB 16.7 MB/s eta 0:00:02


In [4]:
import pandas as pd
import numpy as np

In [7]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [13]:
train_df=pd.DataFrame(dataset["train"])
test_df=pd.DataFrame(dataset["test"])

In [14]:
train_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [15]:
concat_df = pd.concat([train_df, test_df], ignore_index=True)

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
train_df.shape
test_df.shape
concat_df.shape

(8530, 2)

(1066, 2)

(9596, 2)

In [21]:
concat_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [22]:
#doing train test split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = concat_df['text']
y = concat_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, lower=True, split=' ')
tokenizer.fit_on_texts(X_train.values)
X_train = tokenizer.texts_to_sequences(X_train.values)
X_train = pad_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test.values)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
input_length = X_train.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

D:\ProgramFiles\Miniconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test), verbose=2)
model.save('sentiment_model.h5')

Epoch 1/30
120/120 - 6s - 51ms/step - accuracy: 0.9406 - loss: 0.1585 - val_accuracy: 0.7536 - val_loss: 0.7386
Epoch 2/30
120/120 - 7s - 56ms/step - accuracy: 0.9527 - loss: 0.1259 - val_accuracy: 0.7422 - val_loss: 0.7687
Epoch 3/30
120/120 - 7s - 56ms/step - accuracy: 0.9639 - loss: 0.1018 - val_accuracy: 0.7464 - val_loss: 0.9730
Epoch 4/30
120/120 - 7s - 55ms/step - accuracy: 0.9633 - loss: 0.0923 - val_accuracy: 0.7391 - val_loss: 0.9497
Epoch 5/30
120/120 - 7s - 60ms/step - accuracy: 0.9767 - loss: 0.0699 - val_accuracy: 0.7427 - val_loss: 1.0662
Epoch 6/30
120/120 - 7s - 55ms/step - accuracy: 0.9808 - loss: 0.0523 - val_accuracy: 0.7333 - val_loss: 1.1673
Epoch 7/30
120/120 - 7s - 55ms/step - accuracy: 0.9861 - loss: 0.0434 - val_accuracy: 0.7385 - val_loss: 1.1750
Epoch 8/30
120/120 - 7s - 60ms/step - accuracy: 0.9874 - loss: 0.0375 - val_accuracy: 0.7323 - val_loss: 1.2242
Epoch 9/30
120/120 - 7s - 55ms/step - accuracy: 0.9871 - loss: 0.0372 - val_accuracy: 0.7359 - val_loss:

In [41]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model

def predict_sentiment(text, model, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=200)
    pred = model.predict(padded)
    return "Positive" if pred > 0.5 else "Negative"

model = load_model('./sentiment_model.h5')
sample_text = "I am positive!"
sentiment = predict_sentiment(sample_text, model, tokenizer)
print(f'Sentiment: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Sentiment: Positive
